# Interactive Face Detection Sample

This sample showcases Object Detection task applied for face recognition using sequence of neural networks.
Async API usage can improve overall frame-rate of the application, because rather than wait for inference to complete,
the application can continue operating on the host while accelerator is busy.
This sample maintains three parallel infer requests for the Age/Gender Recognition, Head Pose Estimation, and Emotions Recognition that run simultaneously.

Other sample objectives are:

*	Video as input support via OpenCV
*	Visualization of the resulting face bounding boxes from Face Detection network
*	Visualization of age/gender, head pose and emotion information for each detected face

OpenCV is used to draw resulting bounding boxes, labels, and other information. You can copy and paste this code without pulling Inference Engine sample helpers into your application

## How it Works

*	The application reads command line parameters and loads up to four networks depending on `-m...` options family to the Inference Engine.
*	The application gets a frame from the OpenCV's VideoCapture.
*	The application performs inference on the frame detection network.
*	The application performs three simultaneous inferences, using the Age/Gender, Head Pose and Emotions detection networks if they are specified in command line.
*	The application displays the results.

The new Async API operates with a new notion of the Infer Request that encapsulates the inputs/outputs and separates scheduling and waiting for result. For more information about Async API and the difference between Sync and Async modes performance, refer to **How it Works** and **Async API** sections in [Object Detection SSD, Async API Performance Showcase Sample](@ref InferenceEngineObjectDetectionSSDDemoAsyncApplication).


## Compiling

There are two source files: 
* [detectors.cpp](detectors.cpp) -- contains the various helper functions
* [main.cpp](main.cpp) -- contains the main loop

We have provided a Makefile for compiling the application. Run the following cell to run the make command.

In [1]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

In [11]:
!mkdir build && cd build && . ../scripts/setupenv.sh && cmake ..

-- The C compiler identification is GNU 5.4.0
-- The CXX compiler identification is GNU 5.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- InferenceEngine_DIR=/opt/intel//computer_vision_sdk_2018.5.445/deployment_tools/inference_engine/share
-- CMAKE_MODULE_PATH=/opt/intel//computer_vision_sdk_2018.5.445/deployment_tools/inference_engine/share/../samples/cmake
-- Looking for inference engine configuration file at: /home/u25335/Reference-samples/iot-devcloud/cpp/share
-- /etc/*-release distrib: Ubuntu 16.04
-- Found InferenceEngine: /opt

In [12]:
!cd build && make

Scanning dependencies of target gflags_nothreads_static
[  2%] Building CXX object gflags/CMakeFiles/gflags_nothreads_static.dir/src/gflags.cc.o
[  5%] Building CXX object gflags/CMakeFiles/gflags_nothreads_static.dir/src/gflags_reporting.cc.o
[  7%] Building CXX object gflags/CMakeFiles/gflags_nothreads_static.dir/src/gflags_completions.cc.o
[ 10%] Linking CXX static library ../intel64/Release/lib/libgflags_nothreads.a
[ 10%] Built target gflags_nothreads_static
Scanning dependencies of target format_reader
[ 12%] Building CXX object format_reader/CMakeFiles/format_reader.dir/format_reader.cpp.o
[ 15%] Building CXX object format_reader/CMakeFiles/format_reader.dir/bmp.cpp.o
[ 17%] Building CXX object format_reader/CMakeFiles/format_reader.dir/opencv_wraper.cpp.o
[ 20%] Building CXX object format_reader/CMakeFiles/format_reader.dir/MnistUbyte.cpp.o
[ 22%] Linking CXX shared library ../intel64/Release/lib/libformat_reader.so
[ 22%] Built target format_reader
Scanning dependencies of tar

This produces an executable [face_detector](). This executable takes in a number of different command line arguments.

Run the following cell to see the list: 

In [14]:
!cp build/intel64/Release/smart_city_tutorial .

In [1]:
%%bash
export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/data/reference-sample-data/extension/ 
./smart_city_tutorial -h

bash: line 2: ./smart_city_tutorial: No such file or directory


CalledProcessError: Command 'b'export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/data/reference-sample-data/extension/ \n./smart_city_tutorial -h\n'' returned non-zero exit status 127

The version of the cpp file here is a slightly modified version of the faca_detector code built-in to the the Intel® Distribution of OpenVINO™ toolkit.
In this version, the result is written into a output mp4 file specified by the `-o` flag. 

### Running the inference

Now we are ready to run the inference workload. In this step we will be submitting the workload as a job to the job queue.

Currently, you are on what is called a "devnode". On this system, you are allocated just one core on a large Intel® Xeon® CPU. The purpose of this node is to develop code on the devnode and run minimal sections of Jupyter* Notebooks, but it is not meant for compute intensive jobs like deep learning inference. So we need to request additional resources from the cluster of Edge nodes to run the inference, and this is done through the job queue.

To put an item on the job queue, we must first create a bash script that run the workload we want. Run the following cell to create bash script [face_detection_demo.sh](face_detection_demo.sh) which will be our job script. 

In [ ]:
%%writefile face_detection_demo.sh
#PBS
DEVICE=$2
FP_MODEL=$3

if [ "$2" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/altera/aocl-pro-rte/aclrte-linux64/
    source /opt/fpga_support_files/setup_env.sh
    aocl program acl0 /opt/intel/computer_vision_sdk/bitstreams/a10_vision_design_bitstreams/5-0_PL1_FP11_ResNet.aocx
fi

cd $PBS_O_WORKDIR
export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/intel/computer_vision_sdk/deployment_tools/inference_engine/samples/build/intel64/Release/lib/ 
MODEL_ROOT=/opt/intel/computer_vision_sdk/deployment_tools/intel_models
./smart_city_tutorial -i data/video82.mp4 \
-m_vp ${MODEL_ROOT}/person-vehicle-bike-detection-crossroad-0078/${FP_MODEL}/person-vehicle-bike-detection-crossroad-0078.xml \
-d_vp ${DEVICE} -n_async 16 -tracking -collision -no_show

To put this script on the job queue, we use the command `qsub`.
There are two important arguments we use with this command.

First, the `-l` flag.
This flag is used to specify what type of resources to request from the cluster.
For example this can be used to request an Intel® Xeon® CPU based system, or it can be used to request a system with an FPGA accelerator card in it.
The syntax is `-l nodes=1:<tag>` where `<tag>` is the descriptor tag for the resource you want.
For example, `-l nodes=1:iei-tank-xeon` will request an Intel® Xeon® system.
To see the list of available tags, and the number of avilable systems, run the following cell.

In [16]:
!pbsnodes | grep compnode | sort | uniq -c

     49      properties = compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,8gb,1gbe
     11      properties = compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,8gb,1gbe,hddl-f,iei-mustang-f100-a10
     15      properties = compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,8gb,1gbe,hddl-r,iei-mustang-v100-mx8
      4      properties = compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,8gb,1gbe,ncs,intel-ncs
     12      properties = compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,8gb,1gbe,ncs,intel-ncs2
     10      properties = compnode,iei,tank-870,intel-core,i5-7500t,kaby-lake,intel-hd-630,8gb,1gbe
     14      properties = compnode,iei,tank-870,intel-xeon,e3-1268l-v5,skylake,intel-hd-p530,32gb,1gbe
      1      properties = compnode,jwip,intel-atom,e3950,apollo-lake,intel-hd-505,4gb,1gbe
      1      properties = compnode,jwip,intel-core,i5-7500,kaby-lake,intel-hd-630,8gb,1gbe
     15      properties = compnode,

Then there is the `-F` flag, which is used to pass in arguments to the job script.
The [face_detection_demo.sh](face_detection_demo.sh) takes in 2 arguments:
1) the path to the video to run inference on
2) targeted device (CPU,GPU,MYRIAD)
The job scheduler will use the contents of `-F` flag as the argument to the job script.

The following line will request an Intel Xeon system, and passes in "faces-recognition-walking.mp4 CPU" to the job script. Run the cell to submit this job. 

#### Submitting to an edge compute node with an Intel® Core™ CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel 
    Core i5-6500TE</a>. The inference workload will run on the CPU.

In [ ]:
print("Submitting job to Intel Core CPU...")
#Submit job to the queue
job_id_core = !qsub smart_city_demo.sh -l nodes=1:iei-tank-core -F "CPU FP32"
print(job_id_core[0])

#### Submitting to an edge compute node with Intel® Xeon® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel 
    Xeon Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
print("Submitting job to Intel Xeon CPU...")
#Submit job to the queue
job_id_xeon = !qsub smart_city_demo.sh -l nodes=1:iei-tank-xeon -F "CPU FP32"
print(job_id_xeon[0])

Here you should see a output like "{job_id}.c003", where {job_id} is a number.
This is your job ID, and this value can be used to check on the progress of the job down the line.
Furthermore, the above job script has been written so that it uses its job_id as the name of the output video.   

One bigadvantage of the Job queue system is that you may submit multiple jobs at once. 
These jobs will be run as soon as resources are available, and may all run at once if the cluster is not busy.
Here are few other "preset" jobs that for your convenience that you can run. 

#### Submitting to an edge compute node with Intel® Core™ CPU and using the onboard Intel GPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [ ]:
print("Submitting job to Intel Core CPU with Intel GPU...")
#Submit job to the queue
job_id_gpu = !qsub smart_city_demo.sh -l nodes=1:iei-tank-core -F "GPU FP32"
print(job_id_gpu[0])

#### Submitting to an edge compute node with  IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a> . The inference workload will run on the <a href="https://www.ieiworld.com/mustang-f100/en/"> IEI Mustang-F100-A10 </a> card installed in this node.

In [ ]:
print("Submitting job to node with Intel FPGA HDDL-F...")
#Submit job to the queue
job_id_fpga = !qsub smart_city_demo.sh -l nodes=1:iei-tank-fpga -F "HETERO:FPGA,CPU FP32"
print(job_id_fpga[0])

#### Submitting to an edge compute node with Intel® Movidius™ Neural Compute Stick
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://software.intel.com/en-us/movidius-ncs">Intel 
    Movidius Neural Compute Stick</a> installed in this node.

In [ ]:
print("Submitting job to node with Intel Movidius NCS...")
#Submit job to the queue
job_id_ncs = !qsub smart_city_demo.sh -l nodes=1:tank-870:i5-6500te:intel-ncs -F "MYRIAD FP16"
print(job_id_ncs[0])

#### Submitting to an edge compute node with Intel® Movidius™ Neural Compute Stick 2
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
print("Submitting job to node with Intel NCS2...")
#Submit job to the queue
job_id_ncs2 = !qsub smart_city_demo.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "MYRIAD FP16"
print(job_id_ncs2[0])

#### Submitting to an edge compute node with  IEI Mustang-V100-MX8 ( Intel® Movidius™ Myriad™ X Vision Processing Unit (VPU))
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://www.ieiworld.com/mustang-v100/en/">IEI Mustang-V100-MX8 </a>accelerator installed in this node.

In [ ]:
#Submit job to the queue
job_id_vpu = !qsub smart_city_demo.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-v100-mx8 -F "HDDL FP16"
print(job_id_vpu[0])

#### Submitting to an edge compute node with UP Squared Grove IoT Development Kit (UP2)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/up-squared-grove-dev-kit">UP Squared Grove IoT Development Kit</a> edge node with an <a 
    href="https://ark.intel.com/products/96488/Intel-Atom-x7-E3950-Processor-2M-Cache-up-to-2-00-GHz-">Intel Atom® x7-E3950 Processor</a>. The inference  workload will run on the integrated Intel® HD Graphics 505 card.

In [ ]:
#Submit job to the queue
job_id_up2 = !qsub smart_city_demo.sh -l nodes=1:up-squared -F "GPU FP32"
print(job_id_up2[0])

The `qstat` command is used to track the progress of the jobs. 
We have provided a utility funtion "liveQstat()" that provide a live updating GUI for you.
Run the following cell to check on the progress of your earlier jobs.

In [ ]:
liveQstat()

Once the jobs are done. The following cell can be used to display the output. 
The videoHTML is a utility function provided in [demoutils.py](demoutils.py).
This takes one argument, which is the path to the video (see above for convention for the path name).

For your convenience we have stored the jobid from the above `qsub` commands, and the following cells 

In [ ]:
videoHTML('IEI Tank (Intel Core CPU)', 
          ['results/output_' + job_id_core[0] + '.mp4'], 
          'results/stats_'+job_id_core[0]+'.txt')

In [ ]:
videoHTML('IEI Tank Xeon (Intel Xeon CPU)',
          ['results/output_'+job_id_xeon[0]+'.mp4'],
          'results/stats_'+job_id_xeon[0]+'.txt')

In [ ]:
videoHTML('IEI Intel GPU (Intel Core + Onboard GPU)', 
          ['results/output_'+job_id_gpu[0]+'.mp4'],
          'results/stats_'+job_id_gpu[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)',
          ['results/output_'+job_id_fpga[0]+'.mp4'],
          'results/stats_'+job_id_fpga[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + Intel CPU + Intel Movidius NCS',
          ['results/output_'+job_id_ncs[0]+'.mp4'],
          'results/stats_'+job_id_ncs[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + Intel CPU + Intel NCS2',
          ['results/output_'+job_id_ncs2[0]+'.mp4'],
          'results/stats_'+job_id_ncs2[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + IEI Mustang-V100-MX8 (Intel® Movidius™ Myriad™ X Vision Processing Unit (VPU))',
          ['results/output_'+job_id_vpu[0]+'.mp4'],
          'results/stats_'+job_id_vpu[0]+'.txt')

In [ ]:
videoHTML('UP Squared Grove IoT Development Kit (UP2)',
          ['results/output_'+job_id_up2[0]+'.mp4'],
          'results/stats_'+job_id_up2[0]+'.txt')

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('fpga', ' IEI Mustang\nF100-A10\nFPGA'),
             ('ncs', 'Intel\nMovidius\nNCS'),
             ('ncs2', 'Intel\nNCS2'),
             ('vpu', ' IEI Mustang\nV100-MX8\nVPU'),
             ('up2', 'Intel Atom\nx7-E3950\nUP2/GPU')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append(('results/stats_'+vars()['job_id_'+arch][0]+'.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))

summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time' )
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps' )